<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/ISOTIPO-Color.jpg" title="Title text" width="20%" />

<hr style="height:2px;border:none"/>
<H1 align='center'> Challenge Kaggle </H1>
<H5 align='center'> Predicción del precio de una casa: Solución</H5>



<H3> INF-395 Redes Neuronales y Deep Learning </H3>
<H3> Autor: Francisco Andrades</H3>

Lenguaje: Python

Temas:

    - Arquitectura de Redes Neuronales *Feed-Forward*
    - Entrenamiento de Redes Neuronales. 
    - Parte Básica de Redes Convolucionales. 
    - Problemas Especiales.

Link Competencia: https://www.kaggle.com/c/prediccin-del-precio-de-una-casa

Equipo: Bios
<hr style="height:2px;border:none"/>

### Contexto

Queremos predecir el precio de una casa a partir de metadata como el número de habitaciones, el número de baños, el área total de la casa y el zipcode.

### Dataset

El dataset de entrenamiento incluye la información de 443 propiedades con sus respectivos precios. El dataset de pruebas incluye la información de 92 propiedades, pero no estarán disponibles los precios.

**La métrica de evaluación para esta competencia es MSE** 

In [20]:
# Librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import mean_squared_error as mse_loss
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
path = '../input/data-t1-nns/'

In [21]:
#se leen los datos
df_train = pd.read_csv(path+'metadata_casas_train.csv',index_col=0)
df_test = pd.read_csv(path+'metadata_casas_test.csv',index_col=0)
df_train.head()

,cod_casa,habitaciones,banos,area,zipcode,precio
0,OC4U9J6C11R5715,4,4.0,4053,85255,869500
1,CQF44W2LE3S3NSC,4,3.0,3343,36372,865200
2,LT2Z9MOPF5KIM7T,3,4.0,3923,85266,889000
3,VUCNPXL2QMIA22A,5,5.0,4022,85262,910000
4,9CWP6T0SBBLGDBL,3,4.0,4116,85266,971226


In [22]:
#para testing
df_train,df_test = train_test_split(df_train,shuffle=True,random_state=0)

In [23]:
# se realiza la codificación de los zipcodes mencionada en el notebook EDA
def process_metadata(df,lista_columnas,flag):
    df_new = df.copy()
    
    codes = df_new.zipcode
    for n_digits in range(3):
        new_codes = [int(str(i)[:(2*n_digits)+1]) for i in codes.values]
        df_new[str(n_digits)+'_codes'] = new_codes
        one_hot = pd.get_dummies(df_new[str(n_digits)+'_codes'],prefix=str(n_digits))
        df_new = df_new.join(one_hot)
        df_new.drop(str(n_digits)+'_codes',axis=1,inplace=True)
    df_new.drop('zipcode',axis=1,inplace=True)
    if flag:
        columnas_test = []
        for columna in df_new.columns:
            if columna in lista_columnas:
                columnas_test.append(columna)
        df_new = df_new.loc[:,columnas_test]
        df_empty = pd.DataFrame(columns = lista_columnas)
        df_new = df_empty.append(df_new).fillna(0)
    
    return df_new,df_new.columns

df_train_clean1,lista_columnas = process_metadata(df_train,[],0)
df_test_clean1,_ = process_metadata(df_test,lista_columnas,1)

In [24]:
# se elimina la variabla cod_casa que no tiene utilidad (es un id)
df_train_clean1.drop('cod_casa',axis=1,inplace=True)
df_test_clean1.drop('cod_casa',axis=1,inplace=True)

In [25]:
# No se le hace ninguna transformación al precio. Eliminar los outliers más severos resultó
# en una mejor performance en nuestros datos de testing, pero peor en la competencia. Esto
# no es ninguna sorpresa, simplemente quiere decir que existen datos en la competencia que 
# están relacionados con los outliers más severos.

# Trabajar realizando una transformación logarítmica o un reemplazo con algún valor estratégico
# (media/mediana/valor_máximo) resultó en un pero desempeño.
y_train = df_train_clean1['precio']
y_test = df_test_clean1['precio']

X_train = df_train_clean1.drop(['precio'],axis=1)
X_test = df_test_clean1.drop(['precio'],axis=1)

In [26]:
# Se escalan los datos.
scaler_data = StandardScaler()
X_train.iloc[:,:3] = scaler_data.fit_transform(X_train.iloc[:,:3])
X_test.iloc[:,:3] = scaler_data.transform(X_test.iloc[:,:3])

scaler_precio = StandardScaler()
y_train = scaler_precio.fit_transform(np.array(y_train).reshape(-1,1))
y_test = scaler_precio.transform(np.array(y_test).reshape(-1,1))

In [27]:
# Se define modelo Feed-Forward densa con sólo 1 capa oculta con 4 neuronas
# Este modelo se obtuvo mediante grid search  y tuning
# Es el modelo que mejor se comportó 

n_features = X_train.shape[1]
inputs = keras.Input(shape=(n_features, ), name='Input_metadata')
x = layers.Dense(4, activation='relu')(inputs)
outputs = layers.Dense(1, name='Output')(x)
 
model = keras.Model(inputs=inputs, outputs=outputs, name='Modelo_metadata')
model.summary()

Model: "Modelo_metadata"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_metadata (InputLayer)  [(None, 72)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 292       
_________________________________________________________________
Output (Dense)               (None, 1)                 5         
Total params: 297
Trainable params: 297
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Se entrena el modelo por 200 epochs, este número se tuneó poco. Dentro del vecindario [150,250] hay opciones competitivas
model.compile(loss=root_mean_squared_error)
history = model.fit(
        x = X_train,
        y = y_train,
        epochs=200,
        verbose=0,
    ) 
preds = model.predict(X_test)

In [33]:
#para testing
#Error de 300.000 aprox en un rango de valores [0,6.000.000]. Es un error relativamente pequeño.
#Los outliers hacen mucho daño, eliminando los más severos el error disminuye considerablemente.
mse(scaler_precio.inverse_transform(y_test.squeeze()),scaler_precio.inverse_transform(preds),squared=False)

280392.11187145987

In [30]:
#Ignorar, esto es para la competencia
#submission = pd.read_csv('../input/sample-submission/sampleSubmission.csv')
#submission['\'value\''] = scaler_precio.inverse_transform(preds)
#submission.to_csv('submission.csv',index=False)
#submission